In [ ]:
!pip install \
    "pinecone-client[grpc]" \
    "langchain-pinecone" \
    "langchain-openai" \
    "langchain-text-splitters" \
    "langchain"


In [26]:
%env PINECONE_API_KEY = #enter your pinecone api key
%env OPENAI_API_KEY = #enter your open ai api key

env: PINECONE_API_KEY=26ed21f8-e8a2-42db-8ef0-cbc0a437a81f
env: OPENAI_API_KEY=sk-proj-K71bz4rJYXMq9Ipj6RiUT3BlbkFJ6B1Ny92ZukbfNPxRQChw


In [ ]:
!pip install langchain-huggingface
!pip install sentence_transformers

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key= os.environ['PINECONE_API_KEY'])

index_name = "docs-rag-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="docs-rag-chatbot",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [17]:
import json

answers = []
with open('/content/Ecommerce_FAQ_Chatbot_dataset.json', 'r') as f:
  data = json.load(f)
  for item in data["questions"]:
    answers.append(item['answer'])



In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Snowflake/snowflake-arctic-embed-s"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [12]:
print(type(embeddings))

<class 'langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings'>


In [28]:
from langchain_pinecone import PineconeVectorStore


vector_store = PineconeVectorStore.from_texts(
   texts = answers,
   embedding = embeddings,
   index_name = "docs-rag-chatbot",
   namespace = "ecommerce"
)

In [32]:
#object to retrieve information
query = "Can I get a product demo before purchase?"
vector_store.similarity_search(query = query, k = 5, namespace = "ecommerce")

[Document(page_content='We do not currently offer product demonstrations before purchase. However, you can find detailed product descriptions, specifications, and customer reviews on our website.'),
 Document(page_content="Products listed as 'coming soon' are not available for immediate purchase. Please sign up for notifications to be informed when the product becomes available."),
 Document(page_content="If a product is listed as 'coming soon' but not available for pre-order, you will need to wait until it is officially released and becomes available for purchase."),
 Document(page_content="If a product is listed as 'coming soon' but not available for pre-order, you will need to wait until it is officially released and becomes available for purchase."),
 Document(page_content='If a product is not listed on our website, it may not be available for purchase. We recommend exploring the available products or contacting our customer support team for further assistance.')]

In [38]:
# from langchain.chains import RetrievalQA
# from langchain_openai import ChatOpenAI
# import os

# qa_chain = RetrievalQA.from_chain_type(
#     llm = ChatOpenAI(temperature = 0.2, openai_api_key = os.environ["OPENAI_API_KEY"]),
#     chain_type = "stuff",
#     retriever = vector_store.as_retriever(search_kwargs = {"k": 5})
# )
# query = "my product"
# print(qa_chain.invoke({"query": query}).get("result"))

I'm sorry to hear that your product is damaged. Please contact our customer support team immediately for assistance with the necessary steps for repair or replacement.


In [50]:
# from langchain.chains import ConversationalRetrievalChain
# from langchain_openai import ChatOpenAI
# from langchain.memory import ConversationBufferMemory

from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(temperature=0.2, model_name="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])
retriever  = vector_store.as_retriever(search_kwargs = {"k": 5})
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key = 'answer')


# Create the multipurpose chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    retriever=retriever,
    return_source_documents=True
)



In [51]:
qa_chain("My product is damaged")

{'question': 'My product is damaged',
 'chat_history': [HumanMessage(content='My product is damaged'),
  AIMessage(content="I'm sorry to hear that your product is damaged. Please contact our customer support team immediately for assistance with the necessary steps for repair or replacement.")],
 'answer': "I'm sorry to hear that your product is damaged. Please contact our customer support team immediately for assistance with the necessary steps for repair or replacement.",
 'source_documents': [Document(page_content='If you receive a damaged product, please contact our customer support team immediately. We will assist you with the necessary steps for repair or replacement.'),
  Document(page_content='If your product was damaged during shipping, please contact our customer support team immediately. We will guide you through the return and replacement process.'),
  Document(page_content='If your product was damaged due to mishandling during shipping, please contact our customer support t